# Report: CSP and Probabilistic Reasoning for Minesweeper

## Introduction
The goal of this project is to design and evaluate an intelligent agent capable of solving the game of Minesweeper using CSP and PB paradigms. Minesweeper is a puzzle game that combines elements of constraint satisfaction, probabilistic reasoning, and search. Due to its inherent uncertainty and combinatorial complexity, it serves as a valuable testbed for AI strategies.

This work implements and compares five different setups of agents:
- **Random Agent**
- **Backtracking agent: an agent that uses plain Backtracking algorithm**
- **Advanced agent: Backtracking + Minimum Remaining Values (MRV) with Degree Heuristic and Least Constraining Value (LCV)**
- **GAC3 agent: an Advanced agent that uses also Generalized Arc Consistency algorithm**
- **PB agent: an agent that uses all the techniques above + Probabilistic Reasoning when inference isn't enough**

The performance of the agents is assessed in terms of win rate, average number of revealed cells, average time per game.

## Related Works
The Minesweeper problem can be modeled as a Constraint Satisfaction Problem (CSP). In the literature, common techniques include:

- Backtracking with heuristics (such as MRV, LCV, and Degree heuristic) to reduce the search space.

- Forward Checking and Arc Consistency, which help prune inconsistent assignments early, improving efficiency compared to plain backtracking.

However, Minesweeper inherently involves ambiguous situations where pure logical deduction is insufficient. In such cases, we propose the use of probabilistic reasoning, estimating the likelihood that a given cell contains a mine and selecting the safest move accordingly.

In summary, related works highlight a combination of deterministic CSP approaches and probabilistic methods, which provide the theoretical foundation for the agents developed in this project.

## Methodologies


## Probabilistic reasoning module
This module replaces random decisions when constraint-based logic (CSP/AC) cannot prove a certainly-safe move. Given the current knowledge grid (revealed numbers, flagged mines, and unknown cells), it estimates, for every admissible unknown cell, the probability of containing a mine and proposes the minimum-risk action.

## Frontier and constraint model
For each revealed numbered cell with value $k$, consider its 8-neighborhood. Let $\mathcal{U}_{r,c}$ be the set of unknown neighbors of $(r,c)$, and let $\text{KM}_{r,c}$ be the count of already-known adjacent mines (including flagged ones). The local constraint is:
$$
\sum_{v \in \mathcal{U}_{r,c}} X_v \;=\; \max\!\bigl(0,\;\min\bigl(k - \text{KM}_{r,c},\;|\mathcal{U}_{r,c}|\bigr)\bigr),
$$
where each $X_v \in \{0,1\}$ indicates whether cell $v$ is a mine. The *frontier* is the union of unknown cells that appear in at least one such constraint. We build a variable graph by connecting two unknown cells if they co-occur in at least one constraint; its connected components are solved independently and later merged into a single risk map.

## Inference on each component
If a component’s size is below a configurable threshold (default $\mathtt{max\_vars\_exact}=22$), we perform exact inference by enumeration: among all assignments that satisfy every local constraint in the component, the marginal probability for cell $v$ is the fraction of valid assignments where $X_v=1$:
$$
P(X_v=1 \mid \text{evidence}) \;=\; \frac{\#\{\text{valid assignments with } X_v=1\}}{\#\{\text{valid assignments}\}}.
$$
If no valid assignment exists (inconsistency), the component falls back to the approximate scheme described below rather than failing.

## Global priors and “outside frontier” mass
Let $U$ be the set of admissible unknown cells (excluding flagged), and let $M_{\text{rem}}$ be the remaining mines if the total mine count is known; the *global base rate* prior is
$$
p_0 \;=\; \frac{M_{\text{rem}}}{|U|}\quad\text{(or a neutral default if $M_{\text{rem}}$ is unknown).}
$$
After solving frontier components (exact or approximate), let $F$ be the set of frontier variables and let $E_F=\sum_{v\in F} \hat p(v)$ be their expected mine mass. For cells *outside* the frontier, a consistent base rate is assigned by redistributing the remaining mass:
$$
p_{\text{out}} \;=\; \mathrm{clip}_{[0,1]}\!\left( \frac{\max(0,\,M_{\text{rem}} - E_F)}{\max(1,\,|U\setminus F|)} \right).
$$
These priors ensure isolated unknown cells (not adjacent to any number) receive a sane probability even without local constraints.

## Local “pressure” refinement for large components
When a component is too large for exact enumeration, each cell $v$ is refined from the prior using a local *pressure* estimate derived from the numbered neighbors it touches. For every numbered neighbor $s$ around $v$, let $\text{need}_s=\max(0,\,k_s-\text{KM}_s)$ be the remaining mines required by $s$, and let $\text{unk}_s$ be the set of its currently unknown neighbors. The local pressure for $v$ is the mean of per-constraint ratios:
$$
\text{lp}(v) \;=\; \frac{1}{|\mathcal{S}_v|}\sum_{s\in \mathcal{S}_v} \frac{\text{need}_s}{\max(1,\,|\text{unk}_s|)}.
$$
The prior and the local pressure are then blended via a convex combination
$$
\hat p(v) \;\leftarrow\; (1-\alpha)\,\hat p(v) \;+\; \alpha\,\text{lp}(v), \qquad \alpha\in(0,1)\ \text{(default } \alpha=0.7\text{)},
$$
and the result is clamped to $[0,1]$. If a cell has no numbered neighbors, $\text{lp}(v)$ is undefined and the prior is kept.

## Soft calibration to match the remaining mine budget
Cells solved exactly are treated as *fixed-mass*; cells solved approximately are *flexible*. Let $E_{\text{exact}}=\sum_{v\in E}\hat p(v)$ be the mass on exact-inferred cells and $E_{\text{flex}}=\sum_{v\in \mathrm{Flex}}\hat p(v)$ on flexible ones. The flexible mass is softly rescaled toward the budget that remains after exact cells:
$$
E_{\text{target}} \;=\; \max(0,\,M_{\text{rem}} - E_{\text{exact}}),\qquad
s \;=\; \frac{E_{\text{target}}}{\max(\varepsilon,\,E_{\text{flex}})}.
$$
If $|E_{\text{flex}}-E_{\text{target}}|$ exceeds a small tolerance, each flexible $\hat p(v)$ is multiplied by $s$ and re-clamped to $[0,1]$. This preserves exact results while aligning approximate ones with the remaining budget.

## Final risk map and action selection
Merging all components yields a single risk map $\hat p: U\to[0,1]$. The proposed action is the admissible cell with minimal $\hat p(v)$. On ties, we prefer the cell expected to unlock more information when revealed (e.g., more adjacent unknowns constrained by nearby numbers); if still tied, a stable rule resolves the choice. The module never mutates the grid; it returns probabilities and a single recommended move.

## Assumptions and edge cases
Constraints are built from the 8-neighborhood and their right-hand side is clamped to $[0, |\mathcal{U}_{r,c}|]$ for safety; components are independent once split by shared constraints. If a component has no valid assignments under the current evidence, it is treated approximately rather than causing a failure. All probabilities are kept in $[0,1]$.

## Assessment


## Conclusion
